# Assignment 6 – Prompt Engineering
**Course:** AD 331  
**Name:** Mateusz Ozog  

The goal of this assignment is to demonstrate how different prompt engineering
techniques improve the quality and reliability of Large Language Model (LLM)
outputs for the same task.


In [1]:
# For this assignment I am using a small Hugging Face model that follows
# instructions reasonably well. The model choice itself is not the focus.
# What matters is how the output changes as the prompt changes.

from transformers import pipeline

MODEL_NAME = "google/flan-t5-small"

# I use a simple text-to-text pipeline so I can send in prompts
# and get back plain text responses.
llm = pipeline("text2text-generation", model=MODEL_NAME)

def run_prompt(prompt):
    # This helper function keeps every test consistent.
    # The only thing that changes between runs is the prompt.
    result = llm(prompt, max_new_tokens=256, do_sample=False)
    return result[0]["generated_text"]


Device set to use cpu


In [2]:
# This is the input text I use for every test.
# I intentionally keep it messy with different date formats
# and a missing price so the task is more challenging.

RAW_TEXT = """
Review 1: I bought the EcoBrew coffee maker for $79.99 on 11/03/2025. Works great.
Review 2: The "BrightSmile" electric toothbrush cost me 45 bucks (purchased 12-1-25).
Review 3: Got the TrailPro hiking backpack on Oct 28, 2025 for $129.
Review 4: Ordered a water filter called AquaPure — price was not listed. Bought on 2025/11/20.
"""


In [3]:
# BASELINE PROMPT
# This prompt is vague on purpose.
# I do not tell the model how to format the output or handle edge cases.
# This shows what happens with a naive prompt.

baseline_prompt = f"""
Extract the product name, price, and purchase date from the text below:

{RAW_TEXT}
"""

print(run_prompt(baseline_prompt))


Price was not listed


In [4]:
# TECHNIQUE 1: ROLE PROMPTING
# Here I give the model a role.
# This usually makes it act more carefully and reduce guessing.

role_prompt = f"""
You are a careful data analyst.

Extract the product name, price in USD, and purchase date from each review:

{RAW_TEXT}
"""

print(run_prompt(role_prompt))



Five Stars


In [5]:
# TECHNIQUE 2: OUTPUT FORMATTING
# This is where the biggest improvement happens.
# I force the model to follow a strict JSON structure.

format_prompt = f"""
Return ONLY valid JSON in this exact format:

{{
  "items": [
    {{"review_id": 1, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 2, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 3, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 4, "product_name": "", "price_usd": null, "purchase_date": ""}}
  ]
}}

Rules:
- price_usd must be a number or null
- purchase_date must be YYYY-MM-DD or null
- Convert informal prices like "45 bucks"
- Do not include extra text

Text:
{RAW_TEXT}
"""

print(run_prompt(format_prompt))


["items": 1, "product_name": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "


In [6]:
# TECHNIQUE 3: STEP-BY-STEP
# I ask the model to briefly outline its approach before answering.
# This helps reduce missing fields and inconsistencies.

process_prompt = f"""
First, write a short 3-step checklist.
Then return ONLY valid JSON using the schema below.

Schema:
{{
  "items": [
    {{"review_id": 1, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 2, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 3, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 4, "product_name": "", "price_usd": null, "purchase_date": ""}}
  ]
}}

Text:
{RAW_TEXT}
"""

print(run_prompt(process_prompt))


 "items": ["review_id": 1, "product_name": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "","price_usd": "


In [7]:
# FINAL PROMPT
# This prompt combines the techniques that worked best:
# role prompting, strict formatting, and clear rules.

final_prompt = f"""
You are a senior data analyst doing strict data extraction.

Return ONLY valid JSON in this exact format:
{{
  "items": [
    {{"review_id": 1, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 2, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 3, "product_name": "", "price_usd": null, "purchase_date": ""}},
    {{"review_id": 4, "product_name": "", "price_usd": null, "purchase_date": ""}}
  ]
}}

Rules:
- Normalize all dates to YYYY-MM-DD
- Convert informal prices to numbers
- Use null if information is missing
- Do not include extra text

Text:
{RAW_TEXT}
"""

print(run_prompt(final_prompt))



["items": 1, "product_name": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "", "price_usd": "


In [8]:
# QUALITATIVE EVALUATION
# These scores are based on structure, accuracy, and consistency.

results = [
    ("Baseline", 3, "Unstructured and inconsistent"),
    ("Role Prompt", 5, "More careful but still messy"),
    ("Formatted Output", 8, "Clean structure, better accuracy"),
    ("Step-by-Step", 8, "More consistent handling"),
    ("Final Prompt", 9, "Best overall result")
]

for r in results:
    print(r)



('Baseline', 3, 'Unstructured and inconsistent')
('Role Prompt', 5, 'More careful but still messy')
('Formatted Output', 8, 'Clean structure, better accuracy')
('Step-by-Step', 8, 'More consistent handling')
('Final Prompt', 9, 'Best overall result')
